# Geospatial Analysis Primer

In this workbook, we'll look at how we can utilize Python to conveniently process and make predictions on geospatial data. We'll show how three separate datasets can be combined to investigate the relationship between 311 calls in Toronto and the rates of short-term rentals.

Learning Objectives:
- Understand how to load and manipulate geospatial data using Python libraries.
- Spatially join datasets to analyze relationships.
- Model the relationship between 311 calls and short-term rentals.
- Visualize the results of the analysis.

## Setup

For this workbook, we will primarily rely on the GeoPandas library for geospatial data manipulation, along with Folium for visualization and Scikit-learn for modeling. Make sure you have these libraries installed in your Python environment.

In [ ]:
!pip install geopandas folium scikit-learn interpret jupyter ipywidgets --upgrade

We will be working with three datasets from the Toronto Open Data portal: 

1. **Toronto Wards**: A dataset containing the boundaries of Toronto's wards, which will help us understand the spatial distribution of data.
2. **311 Service Requests**: A dataset of 311 service requests in Toronto, which includes the location of each request.
3. **Short-Term Rentals**: A dataset of short-term rental listings in Toronto, which includes the location of each listing.

Our first task is to load these datasets into our Python environment. The Toronto Open Data portal is built on CKAN, which provides a convenient API for accessing datasets. We will use the `requests` library to fetch the data and `pandas` to read it into DataFrames.

In [ ]:
import requests

base_url = "https://ckan0.cf.opendata.inter.prod-toronto.ca/api/3/action/package_show"
datasets = ["city-wards", "311-service-requests-customer-initiated", "short-term-rentals-registration"]
resource_number = [11, 1, 1]  # Resource ID for each dataset
file_type = ["geojson", "zip", "csv"]  # Expected file types for each dataset

for dataset, res_num, f_type in zip(datasets, resource_number, file_type):
    response = requests.get(base_url, params={"id": dataset}).json()
    resource = response['result']['resources'][res_num]
    print(f"Dataset: {dataset}, Resource: {resource['url']}")

    # Download the data
    data_response = requests.get(resource['url'])
    with open(f"{dataset}.{f_type}", "wb") as file:
        file.write(data_response.content)

Now that we've downloaded our three datasets, we can load them into our Python environment. We'll use the `geopandas` library to handle the geospatial data and `pandas` for the tabular data. We'll start by loading the 311 service requests dataset:

In [ ]:
import pandas as pd
import geopandas as gpd
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

service_requests = pd.read_csv("311-service-requests-customer-initiated.zip", compression='zip', on_bad_lines='skip', low_memory=False)
to_keep = ["Ward", "Service Request Type", "Division", "Section", "First 3 Chars of Postal Code"]
service_requests = service_requests[to_keep]
# Split the "Ward" column into "Ward Number" and "Ward Name". Format is Ward Name (Number). There are non-alphabet characters in the ward name, e.g. Toronto-St. Paul's (12)
service_requests[['Ward Name', 'Ward Number']] = service_requests['Ward'].str.extract(r'([^\d]+)\s*\((\d+)\)')
# Drop the original "Ward" column and any rows with missing values in "Ward Name" or "Ward Number"
service_requests = service_requests.drop(columns=['Ward']).dropna(subset=['Ward Name', 'Ward Number'])
# Convert "Ward Number" to integer type
service_requests['Ward Number'] = service_requests['Ward Number'].astype(int)
service_requests.head()

We had to do some cleaning there to prepare the data for us. Namely, we split the single "Ward" column, with entries like "Toronto-St. Paul's (12)" into two separate columns: "Ward Number" and "Ward Name". We also dropped any rows that weren't super relevant to our analysis, such as those without a ward number or name, and dropped columns that we didn't need for our analysis.

Next, we can load the rental data. Similarly, we will clean it up by keeping only the relevant columns and renaming them for consistency. We will also ensure that the "Ward Number" is in the correct format for merging with the 311 service requests data:

In [ ]:
rentals = pd.read_csv("short-term-rentals-registration.csv" , on_bad_lines='skip', low_memory=False)
to_keep = ["property_type", "ward_number", "ward_name", "postal_code"]
rentals = rentals[to_keep]
rentals = rentals.rename(columns={"ward_number": "Ward Number", "ward_name": "Ward Name", "postal_code": "First 3 Chars of Postal Code"})
rentals['Ward Number'] = rentals['Ward Number'].astype(int)
rentals = rentals.dropna(subset=['Ward Name', 'Ward Number'])
rentals.head()

Lastly, we will load the Toronto wards data. This dataset contains the geographical boundaries of each ward, which will allow us to spatially join it with the 311 service requests and short-term rentals data.

In [ ]:
wards = gpd.read_file("city-wards.geojson")
to_keep = ["AREA_SHORT_CODE", "AREA_NAME", "geometry"]
wards = wards[to_keep]
wards = wards.rename(columns={"AREA_SHORT_CODE": "Ward Number", "AREA_NAME": "Ward Name"})
wards['Ward Number'] = wards['Ward Number'].astype(int)
wards = wards.dropna(subset=['Ward Name', 'Ward Number'])
wards.head()

GeoPandas not only lets us access the data in a tabular way as if it's a standard Pandas DataFrame, but it also allows us to quickly visualize the data on a map, and perform spatial operations like joins and intersections. Let's visualize the wards to make sure they look how we'd expect:

In [ ]:
import matplotlib.pyplot as plt
# Basic plot of the wards with no background map
wards.plot(figsize=(10, 10), edgecolor='black', alpha=0.5)

# Write ward numbers on top of each ward
for x, y, label in zip(wards.geometry.centroid.x, wards.geometry.centroid.y, wards['Ward Number']):
    plt.text(x, y, label, fontsize=12, ha='center', va='center')

Further along in the process, we'll use `Folium` to produce interactive maps that let us visualize the data in a more user-friendly way. This will allow us to see the distribution of 311 service requests and short-term rentals across the wards of Toronto. For now, though, just seeing the basic plot from GeoPandas is a good start to ensure our data is loaded correctly.

## Combining Datasets

Now that we have our three datasets loaded and cleaned, we can proceed to combine them. The goal is to analyze the relationship between 311 service requests and short-term rentals across the wards of Toronto.

Initially, we'll do this by joining on the ward number. This will allow us to aggregate the 311 service requests and short-term rentals by ward, enabling us to analyze the relationship between these two datasets.

### Aggregate 311 by Service Request Type

To aggregate our service requests, we'll group our DataFrame by both Ward Number and Service Request Type. This will tell us how many service requests of each type were made in each ward. Then, we use the `.size()` method to count the number of requests for each type in each ward, which we'll ultimately be combining with the short-term rentals data.

In [ ]:
agg_311 = (
    service_requests
    .groupby(['Ward Number', 'Service Request Type'])
    .size()
    .unstack(fill_value=0)
    .add_prefix('311_')
    .reset_index()
)

agg_311.head()

### Aggregate STR count by Property Type

Next, we'll apply a similar logic to our short-term rentals data. We'll group by Ward Number and Property Type to get the count of each type of rental in each ward. This will give us a clearer picture of the distribution of short-term rentals across the wards.

In [ ]:
agg_str = (
    rentals
    .groupby(['Ward Number', 'property_type'])
    .size()
    .unstack(fill_value=0)
    .add_prefix('STR_')
    .reset_index()
)

agg_str.head()

Now that we have our aggregate data for each ward, we can merge into the wards GeoDataFrame. This will allow us to visualize the data on a map and perform further analysis.

You'll note that we supply the `on` and `how` parameters to the `merge` function. The `on` parameter specifies the column to join on, which in this case is "Ward Number". The `how` parameter specifies the type of join to perform; we use 'left' to keep all wards even if they don't have any service requests or short-term rentals.

In [ ]:
gdf_combined = wards.merge(agg_311, on='Ward Number', how='left').merge(agg_str, on='Ward Number', how='left')

gdf_combined = gdf_combined.fillna(0)  # Fill NaN values with 0 for better visualization

gdf_combined.head()

We can now trivially plot values for any given column in our combined GeoDataFrame. For example, below we'll plot the number of `311_Complaint - Crossing Guard Conduct` requests in each ward. This will give us a visual representation of where these types of service requests are concentrated across the wards of Toronto.

In [ ]:
gdf_combined.plot(column='311_Complaint - Crossing Guard Conduct', figsize=(10, 10), legend=True, cmap='OrRd', edgecolor='black')
plt.title('311 Complaints - Crossing Guard Conduct by Ward')
plt.axis('off');

For more interactive and frankly better looking maps, we can turn to the `Folium` library. Folium allows us to create interactive maps that can be embedded in Jupyter notebooks or saved as HTML files. This is particularly useful for visualizing geospatial data, as it provides a more user-friendly interface for exploring the data.

In [ ]:
import folium
import branca.colormap as cm

# 1) Build a colour scale based on STR_Apartment range
vmin, vmax = gdf_combined["STR_Apartment"].min(), gdf_combined["STR_Apartment"].max()
colormap = cm.linear.OrRd_09.scale(vmin, vmax)        # orange-red gradient
colormap.caption = "Short-term-rental apartments (count)"

# 2) Start the base map centred on Toronto
m = folium.Map(location=[43.7000, -79.4200], zoom_start=11, tiles="cartodbpositron")

# 3) Add polygons with ward-level styling + hover tool-tip
folium.GeoJson(
    gdf_combined,
    name="STR Apartments",
    style_function=lambda feat: {
        "fillColor": colormap(feat["properties"]["STR_Apartment"])
                     if feat["properties"]["STR_Apartment"] is not None else "#d3d3d3",
        "color": "black",
        "weight": 0.8,
        "fillOpacity": 0.7,
    },
    tooltip=folium.features.GeoJsonTooltip(
        fields=["Ward Name", "STR_Apartment"],
        aliases=["Ward", "STR apartments"],
        localize=True
    ),
).add_to(m)

# 4) Add legend + layer control
colormap.add_to(m)
folium.LayerControl().add_to(m)

# 5) Save and/or display
m.save("toronto_str_apartments_map.html")
m          # in-notebook display

## Modelling

Now, let's use the combined dataset to model the relationship between 311 service requests and short-term rentals. We'll use a Random Forest Regressor from the scikit-learn library to predict the number of 311 service requests (by type) based on the number of short-term rentals in each ward. Then, we can examine to see if any of the short-term rental types are significant predictors of the number of 311 service requests.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, root_mean_squared_error
from tqdm.auto import tqdm

############################################################
# 1. Feature / target matrices
############################################################
# Keep only numeric, non-geometry columns
numeric_df = gdf_combined.drop(columns="geometry")

# Predictor columns  = all STR_* variables
str_cols = [c for c in numeric_df.columns if c.startswith("STR_")]
X = numeric_df[str_cols]

# Target columns     = all 311_* variables
target_cols = [c for c in numeric_df.columns if c.startswith("311_")]

print(f"Predictor columns: {X.columns.tolist()}")
print(f"Target columns: {target_cols}")

In [ ]:
############################################################
# 2. Hyper-parameters for the forest - change if running slowly
############################################################
rf_params = dict(
    n_estimators=500, # Reduce to 100 if running slowly
    max_depth=None,
    min_samples_split=2,
    random_state=42,
)

In [ ]:
############################################################
# 3. Fit one Random Forest per 311 category
############################################################
results = []          # will collect performance metrics
importances = []      # will collect feature importance per model

for target in tqdm(target_cols, desc="Fitting models", unit="model"):
    y = numeric_df[target]

    # Split ward rows 80/20
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42
    )

    # Fit
    rf = RandomForestRegressor(**rf_params)
    rf.fit(X_train, y_train)

    # Predict → test metrics
    y_pred = rf.predict(X_test)
    r2   = r2_score(y_test, y_pred)
    rmse = root_mean_squared_error(y_test, y_pred)

    # Store overall metrics
    results.append({"311_type": target, "R2_test": r2, "RMSE_test": rmse})

    # Store and label feature importances
    imp = pd.Series(rf.feature_importances_, index=X.columns, name=target)
    importances.append(imp.sort_values(ascending=False))


In [ ]:
############################################################
# 4. Show summary table
############################################################
results_df = pd.DataFrame(results).sort_values("R2_test", ascending=False)

# Display columns where R2_test is greater than 0.75
display(results_df[results_df["R2_test"] > 0.75])

Now that our models are complete, we can see there are approximately 90 service request types where the R2 score is greater than 0.5, indicating a strong relationship between the number of short-term rentals and the number of service requests.

We can further investigate whether there is a specific relationship between type of short-term rental and the types of service requests made. Let's utilize the interpret library to build a **Glass Box** model. This is a model that allows us to see the feature importances and understand how each feature contributes to the predictions.

We'll focus on the `311_Litter / Bike Removal Inquiry` service request, which has a high R2 of around 0.975. This indicates a strong relationship between the number of short-term rentals and the number of litter/bike removal inquiries in each ward. Let's see if that holds up under greater scrutiny.

In [ ]:
from interpret import show
from interpret.glassbox import ExplainableBoostingRegressor

y = numeric_df["311_Litter / Bike Removal Inquiry"]
X = numeric_df[str_cols]

ebm = ExplainableBoostingRegressor(random_state=42)
ebm.fit(X, y)
show(ebm.explain_global(name="311_Litter / Bike Removal Inquiry"))

In [ ]:
show(ebm.explain_local(X, y, name="311_Litter / Bike Removal Inquiry"))

## Extension: Postal-Code Level Analysis

Our two tabular datasets also contain the first three letters of the postal code for each service request and short-term rental. This means we can also aggregate our data at the postal code level, which will give us a more granular view of the relationship between short-term rentals and 311 service requests.

Download the FSA dataset below from Statistics Canada to get started:

In [ ]:
import zipfile

fsa_boundaries_url = "https://www12.statcan.gc.ca/census-recensement/2021/geo/sip-pis/boundary-limites/files-fichiers/lfsa000a21a_e.zip"

response = requests.get(fsa_boundaries_url)
with open("fsa_boundaries.zip", "wb") as file:
    file.write(response.content)
with zipfile.ZipFile("fsa_boundaries.zip", 'r') as zip_ref:
    zip_ref.extractall(".")

fsa_boundaries = gpd.read_file("lfsa000a21a_e/lfsa000a21a_e.shp")

In [ ]:
fsa_boundaries.head()

We only need the FSAs that could be present in Toronto, so we can begin by filtering down to that. We can check if the "PRNAME" field contains "Ontario" and the "FSA" field starts with "M", which is the prefix for Toronto postal codes.

In [ ]:
fsa_boundaries = fsa_boundaries[fsa_boundaries["PRNAME"].str.contains("Ontario") & fsa_boundaries["CFSAUID"].str.startswith("M")]

In [ ]:
fsa_boundaries.plot(figsize=(10, 10), edgecolor='black', alpha=0.5)

Great, that looks like Toronto! Now we can break down our data by FSA instead of by the larger wards. This will allow us to see the distribution of 311 service requests and short-term rentals at a more granular level.

Our data directly tells us how many service requests and short-term rentals there are in each FSA, so we can easily aggregate our data by FSA. We can then merge this aggregated data with the FSA boundaries to visualize the data on a map. Let's imagine, though, that we don't have this data and we only have the ward-level information. We can still disaggregate our data by FSA and get an approximation of the number of service requests and short-term rentals in each FSA. Let's explore how we might do that.

### 1. Prep

We first need to quickly make sure that both layers share the same CRS (Coordinate Reference System). This is important for spatial operations like joins and intersections. We can use the `to_crs` method to convert the FSA boundaries to the same CRS as our combined GeoDataFrame.

### 2. Ward x FSA intersection + area fractions

Now that we have both layers in the same CRS, we can perform a spatial join to find the intersection between the wards and FSAs. This will allow us to calculate the area of each intersection, which we can then use to determine the fraction of each ward that falls within each FSA.

### 3. Apportioning service requests and short-term rentals
With the area fractions calculated, we can now apportion the service requests and short-term rentals from the ward level to the FSA level. This will give us a more granular view of the relationship between short-term rentals and 311 service requests at the postal code level.

### 4. Aggregation to FSA level
Finally, we can aggregate the data by FSA to get the total number of service requests and short-term rentals in each FSA. This will allow us to analyze the relationship between short-term rentals and 311 service requests at a more granular level.

In [ ]:
# ----------------------------------------------------------
# 1. Prep – make sure both layers share the same CRS
# ----------------------------------------------------------
fsa_boundaries = fsa_boundaries.to_crs(gdf_combined.crs)

# ----------------------------------------------------------
# 2. Ward x FSA intersection
# ----------------------------------------------------------
ward_fsa = gpd.overlay(gdf_combined, fsa_boundaries, how="intersection")

# Area of each overlap polygon
ward_fsa["area_overlap"] = ward_fsa.geometry.area

ward_area = (
    gdf_combined
      .assign(ward_area=gdf_combined.geometry.area)
      .loc[:, ["Ward Number", "ward_area"]]       
)

ward_fsa = ward_fsa.merge(ward_area, on="Ward Number", how="left")
ward_fsa["area_frac"] = ward_fsa["area_overlap"] / ward_fsa["ward_area"]


# ----------------------------------------------------------
# 3. Vectorised apportioning
# ----------------------------------------------------------
numeric_cols = [
    c for c in gdf_combined.columns
    if c.startswith("311_") or c.startswith("STR_")
]

apportioned = (
    ward_fsa[numeric_cols]
        .mul(ward_fsa["area_frac"], axis=0)   # scale by fraction
        .add_suffix("_apportioned")           # rename once
)

ward_fsa = pd.concat(
    [ward_fsa[["CFSAUID", "geometry"]], apportioned],
    axis=1,
    copy=False
)

    
# ----------------------------------------------------------
# 4. Aggregate to FSAs
# ----------------------------------------------------------
fsa_aggregated = (
    ward_fsa
      .drop(columns="geometry")
      .groupby("CFSAUID")
      .sum()
      .reset_index()
)

fsa_map = (
    fsa_boundaries
      .merge(fsa_aggregated, on="CFSAUID", how="left")
      .fillna(0)
)

# ----------------------------------------------------------
# 5. Quick Folium choropleth (example: STR apartments)
# ----------------------------------------------------------
column = "STR_Apartment_apportioned"

vmin, vmax = fsa_map[column].min(), fsa_map[column].max()
cmap = cm.linear.OrRd_09.scale(vmin, vmax).to_step(n=6)
cmap.caption = "Estimated STR Apartments (area-weighted)"

m = folium.Map(location=[43.7000, -79.4200], zoom_start=11, tiles="cartodbpositron")

folium.GeoJson(
    fsa_map,
    name="STR Apartments (FSA)",
    style_function=lambda feat: {
        "fillColor": cmap(feat["properties"][column]),
        "color": "black",
        "weight": 0.5,
        "fillOpacity": 0.7,
    },
    tooltip=folium.GeoJsonTooltip(
        fields=["CFSAUID", column],
        aliases=["FSA", "Est. STR apartments"],
        localize=True,
    ),
).add_to(m)

cmap.add_to(m)
folium.LayerControl().add_to(m)

m.save("fsa_str_apportioned_map.html")
m  # shows the map inline in a Jupyter notebook

Let's examine how this compares to the real data we have for the FSAs. We can aggregate our original data to the FSA level and plot.

In [ ]:
agg_311 = (
    service_requests
    .groupby(['First 3 Chars of Postal Code', 'Service Request Type'])
    .size()
    .unstack(fill_value=0)
    .add_prefix('311_')
    .reset_index()
)

# Drop the entry where First 3 Chars of Postal Code is "Intersection"
agg_311 = agg_311[agg_311['First 3 Chars of Postal Code'] != 'Intersection']

agg_311.head()

In [ ]:
agg_str = (
    rentals
    .groupby(['First 3 Chars of Postal Code', 'property_type'])
    .size()
    .unstack(fill_value=0)
    .add_prefix('STR_')
    .reset_index()
)

agg_str.head()

In [ ]:
# Rename First 3 Chars of Postal Code to CFSAUID
agg_311 = agg_311.rename(columns={"First 3 Chars of Postal Code": "CFSAUID"})
agg_str = agg_str.rename(columns={"First 3 Chars of Postal Code": "CFSAUID"})

In [ ]:
# Merge the 311 and STR dataframes on CFSAUID with fsa_boundaries
gdf_fsa = fsa_boundaries.merge(agg_311, on="CFSAUID", how="left").merge(agg_str, on="CFSAUID", how="left")

# Fill NaN values with 0 for better visualization
gdf_fsa = gdf_fsa.fillna(0)

# Plot STR_Apartment with Folium

vmin, vmax = gdf_fsa["STR_Apartment"].min(), gdf_fsa["STR_Apartment"].max()
colormap = cm.linear.OrRd_09.scale(vmin, vmax)        # orange-red gradient
colormap.caption = "Short-term-rental apartments (count)"  
# Start the base map centred on Toronto
m_fsa = folium.Map(location=[43.7000, -79.4200], zoom_start=11, tiles="cartodbpositron")
# Add polygons with FSA-level styling + hover tool-tip
folium.GeoJson(
    gdf_fsa,
    name="STR Apartments",
    style_function=lambda feat: {
        "fillColor": colormap(feat["properties"]["STR_Apartment"])
                     if feat["properties"]["STR_Apartment"] is not None else "#d3d3d3",
        "color": "black",
        "weight": 0.8,
        "fillOpacity": 0.7,
    },
    tooltip=folium.features.GeoJsonTooltip(
        fields=["CFSAUID", "STR_Apartment"],
        aliases=["FSA", "STR apartments"],
        localize=True
    ),
).add_to(m_fsa)
# Add legend + layer control
colormap.add_to(m_fsa)
folium.LayerControl().add_to(m_fsa)
# Save and/or display
m_fsa.save("toronto_fsa_str_apartments_map.html")
m_fsa          # in-notebook display